### Agent Reflection
Agent Reflection is a process where an AI agent reviews its own actions, decisions, and outcomes to improve its performance over time. This involves analyzing past interactions, identifying areas for improvement, and making adjustments to strategies or behaviors.

#spanish
### Reflexión del Agente
La Reflexión del Agente es un proceso en el que un agente de IA revisa sus propias acciones, decisiones y resultados para mejorar su rendimiento con el tiempo. Esto implica analizar interacciones pasadas, identificar áreas de mejora y hacer ajustes en estrategias o comportamientos.


In [ ]:
import os 
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_core.messages import SystemMessage, HummanMessage 
from langgraph.graph import StateGraph, MessagesState, START, END 
from typing import Literal, TypedDict, List


load_dotenv()

class ReflectionState(MessagesState):
    draft: str = "" #Borrador actual
    reflection: str = "" #Feedback del critico
    iterations: int = 0 #Numero de iteraciones realizadas
    max_iterations: int = 3 #maximo de iteraciones permitidas 
    

"""Nodo que crea o mejor el contenido"""
def generator_node(state):
    """Generator Node - "El escritor del sistema
    Responsabilidades:
    - Generar contenido inicial 
    - Mejorar Contendio basandose en el feedback 
    - Producir respuesta de calidad 
    
    """
    llm = init_chat_model(model="gpt-4o-mini", model_provider="openai")

    if state.get("reflection"):
        prompt = f"""
         Tarea : Mejorar el siguiente contenido

         CONTENIDO ACTUAL: {state['draft']}
         FEEDBACK RECIBIDO: {state['reflection']}
         SOLICITUD ORIGINAL: {state['messages'][-1].content}

         Genera una version mejorada que aborde todos los puntos del feedback 
        """
    else:
        prompt = f"""
         TAREA: Generar contenido de alta calidad 
         SOLICITUD: {state['messages'][-1].content} 
         Crea una respuesta completa, clara y util
        """

    response = llm.invoke([HummanMessage(content=prompt)])

    return {
        "draft": response.content,
        "iterations": state.get("iterations",0) + 1
    }

def reflection_node(state: ReflectionState):
    """ Reflectio Node - El critico del sistema 
        Responsabilidades :
        - Analizar el contenido generado 
        - Identificar areas de mejora 
        - Proporcionar feedback constructivo
    """
    
    llm = init_chat_model(model="gpt-4o-mini", model_provider="openai")
    
    prompt = f"""
       Eres un critico experto que evalua el contenido 
       
       CONTENIDO A EVALUAR 
       {state['draft']}
       
       SOLICITUD ORIGINAL:
       {state['messages'][-1].content}
       
       Analiza el contenido y proporciona el feedback constructivo
       1- Claridad y estructura 
       2- Completitud de la respuesta 
       3- Precision tecnica 
       4- Areas especificas de mejora 
       
       Si el contenido es excelente, indica "APROBADO - El contenido cumple todos los criterios 
       Si necesitas mejoras, proporciona puntos especificos a mejorar
    """
    
    response = llm.invoke([HummanMessage(content=prompt)])
    
    return {
        "reflection": response.content,
        "messages": state["messages"]
    } 
    
    
def should_continue_reflection(state: ReflectionState) -> Literal["reflect","end"]:
    """ Decide si continuar refinando o terminar
    
    Criterios:
     - Si alcanzo max_iterations -> terminar 
     - Si el feedback indica "Aprobado" ->terminar
     - De lo contrario -> continuar reflexionando  
    """    
    
    #Verifica limite de iteraciones
    if state['max_iterations'] >= state.get("max_iterations", 3):
        print(f"Limite de iteraciones alcanzado ({state['iterations']})")
        return "end"
    
    if state.get("reflection") and "APROBADO" in state["reflection"]:
        print(f"Contenido aprobado en interacion : {state["iterations"]}")
        return "end"
    
    print(f"Iteracion {state['iterations']}: Refinando contenido...")
    return "reflect"

  
graph = StateGraph(ReflectionState)
graph.add_node("generator",generator_node)
graph.add_node("reflection", reflection_node)
graph.add_edge(START, "generator")
graph.add_conditional_edges(
    "generator",
    should_continue_reflection,
    {
        "end":END,
        "reflect":"reflection"
    }
)
graph.add_edge("reflection", "generator")

 
    
    
    


